# Text Classification - RNN

### Importing

In [18]:
import tensorflow_datasets as tfds
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

### Helper Functions

In [6]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

### Loading the Data

In [7]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

2023-02-10 16:19:19.110372: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling /Users/sankeerthana/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAAXK8L/imdb_reviews-…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /Users/sankeerthana/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAAXK8L/imdb_reviews-…

Dataset imdb_reviews downloaded and prepared to /Users/sankeerthana/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-10 16:20:14.041717: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-10 16:20:14.042453: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
print(f"Length of Train Dataset: {len(train_dataset)}, Length of Test Dataset: {len(test_dataset)}")

Length of Train Dataset: 25000, Length of Test Dataset: 25000


### Exploring the Datasets

In [25]:
label_list = []
for record, label in train_dataset:
    label_list.append(int(label))

The dataset is a pair of text and its corresponding label. 0 for negative and 1 for positive.

Need to shuffle it for randomness.

In [28]:
class_division = Counter(label_list)
class_division

Counter({0: 12500, 1: 12500})

The dataset is also balanced as we have equal number of positive and negative samples.

### Creating pairs and shuffling the samples

In [29]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [30]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

.prefetch() method is added to ensure that there is always one batch of data that is available.

In [34]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b"I don't remember when I first heard about this movie, but I rented it about six years ago, and it still remains one of my favorite comedies. I will admit, you probably will despise this movie if you know nothing about rap music. But if you are a rap fan, even a casual one, you will love the inside jokes and references. One of the best lines in the movie is about the difference between a b**** and a h**; I still use this line today and get lots of laughs with it. One of the best performances comes from Larry Scott, who played nerd Lamar in `Revenge of the Nerds'. It is unfortunate that this movie will likely never get a DVD release."
 b"A somewhat dull made for tv movie which premiered on the TBS cable station. Antonio and Janine run around chasing a killer computer virus and...that's about it. For trivia buffs this will be noted as debuting the same weekend that the real life 'Melissa' virus also made it's debut in e-mail inboxes across the world."
 b'While browsing the inte

### Text Encoder 

In [35]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

ModuleNotFoundError: No module named 'keras'